In [1]:
using BenchmarkTools

# Common

In [2]:
struct Point
    x::Float64
    y::Float64
end

# Direction vector
struct DVector
    x::Float64
    y::Float64
end

function cross_product(u::DVector, v::DVector)
    u.x * v.y - u.y * v.x
end

function dot_product(u::DVector, v::DVector)
    u.x * v.x + u.y * v.y
end

dot_product (generic function with 1 method)

# Sutherland-Hodgman

In [3]:
function _inside(p::Point, r::Point, U::DVector)
    U.x * (p.y - r.y) > U.y * (p.x - r.x)
end

function _intersection(a::Point, V::DVector, r::Point, N::DVector)
    W = DVector(r.x - a.x, r.y - a.y)
    nw = dot_product(N, W)
    nv = dot_product(N, V)
    if nv != 0.0
        t = nw / nv
        return true, Point(a.x + t * V.x, a.y + t * V.y)
    else
        return false, Point(0.0, 0.0)
    end
end

function polygon_area(polygon, len)
    area = 0.0
    a = Point(polygon[1, 1], polygon[2, 1])
    b = Point(polygon[1, 2], polygon[2, 2])
    U = DVector(b.x - a.x, b.y - a.y)
    for i in 3:len
        c = Point(polygon[1, i], polygon[2, i])
        V = DVector(a.x - c.x, a.y - c.y)
        area += abs(cross_product(U, V))
        b = c
        U = V
    end
    return 0.5 * area
end

function _copy(src, dst, n)
    for i in 1:n
        dst[1, i] = src[1, i]
        dst[2, i] = src[2, i]
    end
end

function _push(array, n, value)
    array[1, n + 1] = value.x
    array[2, n + 1] = value.y
    return n + 1
end

function clip_polygons(polygon, clipper)
    n_output = size(polygon)[2]
    n_clip = size(clipper)[2]
    output = Array{Float64}(undef, (2, 6))
    subject = Array{Float64}(undef, (2, 6))
    _copy(polygon, output, n_output)
    
    r = Point(clipper[1, n_clip], clipper[2, n_clip])
    for i in 1:n_clip
        s = Point(clipper[1, i], clipper[2, i])

        U = DVector(s.x - r.x, s.y - r.y)
        N = DVector(-U.y, U.x)
        if U.x == 0.0 && U.y == 0.0
            continue
        end
        len = n_output
        _copy(output, subject, len)
        n_output = 0

        a = Point(subject[1, len], subject[2, len])
        a_inside = _inside(a, r, U)
        for j in 1:len
            b = Point(subject[1, j], subject[2, j])

            V = DVector(b.x - a.x, b.y - a.y)
            if V.x == 0.0 && V.y == 0.0
                continue
            end
            b_inside = _inside(b, r, U)
            if b_inside
                if !a_inside
                    succes, point = _intersection(a, V, r, N)
                    if succes
                        n_output = _push(output, n_output, point)
                    end
                end
                n_output = _push(output, n_output, b)
            elseif a_inside
                succes, point = _intersection(a, V, r, N)
                if succes
                    n_output = _push(output, n_output, point)
                else
                    b_inside = true
                    n_output = _push(output, n_output, b)
                end
            end
            
            a = b
            a_inside = b_inside
        end
        
        if n_output < 3
            return 0.0
        end
        
        r = s
    end
    area = polygon_area(output, n_output)
    return area
end      

clip_polygons (generic function with 1 method)

# Generate counter clockwise triangles

In [4]:
function ccw!(a)
    n = size(a)[end]
    
    for i in 1:n
        t = a[:, :, i]
        normal = (t[1, 2] - t[1, 1])*(t[2, 3]-t[2, 1])-(t[2, 2]-t[2, 1])*(t[1, 3]-t[1, 1])

        if normal < 0
            a[:, :, i] .= reverse(t, dims=2)
        end         
    end
end

a = rand(2, 3, 1000000)
b = rand(2, 3, 1000000)
ccw!(a)
ccw!(b)

# Test intersection

In [5]:
function area_of_intersection(a, b)
    n = size(a)[end]
    out = Array{Float64}(undef, n)
    for i in 1:n
        t0 = a[:, :, i]
        t1 = b[:, :, i]
        out[i] = clip_polygons(t0, t1)
    end
    return out
end

area_of_intersection (generic function with 1 method)

In [6]:
@btime area_of_intersection(a, b)

  330.447 ms (4000002 allocations: 587.46 MiB)


1000000-element Array{Float64,1}:
 0.014391781315202726 
 0.0003064186767845136
 0.0002968296136513384
 0.0002866365320717927
 0.0                  
 0.017467793710161897 
 0.009843798894065208 
 0.0                  
 0.0                  
 0.0                  
 0.012743618317396514 
 0.006871623315515474 
 0.0                  
 ⋮                    
 0.00849481186994664  
 0.0007650273034510932
 0.011486340277840412 
 0.0                  
 0.04689887920880241  
 0.00655943463347236  
 0.008974314733621053 
 0.0004920024442981283
 0.0019692347708163273
 0.0                  
 0.0043026069093412955
 0.03881890508003275  